In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [3]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
# Store layers weight & bias

# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'h1': tf.Variable(random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(random_normal([n_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.zeros([num_classes]))
}

In [10]:
# Create model.
def neural_net(x):
  # Hidden fully connected layer with 128 neurons.
  layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  # Apply sigmoid to layer_1 output for non-linearity.
  layer_1 = tf.nn.sigmoid(layer_1)

  # Hidden fully connected layer with 256 neurons.
  layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
  # Apply sigmoid to layer_2 output for non-linearity.
  layer_2 = tf.nn.sigmoid(layer_2)

  # Output fully connected layer with a neuron for each class.
  out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
  # Apply softmax to normalize the logits to a proability distribution.
  return tf.nn.softmax(out_layer)

In [7]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
  # Encode label to a one hot vector.
  y_true = tf.one_hot(y_true, depth=num_classes)
  # Clip prediction values to avoid log(0) error.
  y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
  # Compute cross-entropy.
  return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
  # Predicted class is the index of highest score in prediction vector (i.e. argmax).
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [12]:
# Optimization process.
def run_optimization(x, y):
  # Wrap computation inside a GradientTape for automatic differentiation.
  with tf.GradientTape() as g:
    pred = neural_net(x)
    loss = cross_entropy(pred, y)
  
  # Variables to update, i.e. trainable variables.
  trainable_variables = list(weights.values()) + list(biases.values())

  # Compute gradients.
  gradients = g.gradient(loss, trainable_variables)

  # Update W and b following gradients.
  optimizer.apply_gradients(zip(gradients, trainable_variables))

In [14]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  # Run the optimization to update W and b values.
  run_optimization(batch_x, batch_y)
  
  if step % display_step == 0:
    pred = neural_net(batch_x)
    loss = cross_entropy(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 53.490246, accuracy: 0.953125
step: 200, loss: 48.604042, accuracy: 0.945312
step: 300, loss: 28.754854, accuracy: 0.976562
step: 400, loss: 44.417061, accuracy: 0.964844
step: 500, loss: 53.188457, accuracy: 0.953125
step: 600, loss: 54.863403, accuracy: 0.941406
step: 700, loss: 35.902515, accuracy: 0.953125
step: 800, loss: 39.689758, accuracy: 0.957031
step: 900, loss: 39.051590, accuracy: 0.949219
step: 1000, loss: 43.869492, accuracy: 0.957031
step: 1100, loss: 50.766663, accuracy: 0.953125
step: 1200, loss: 34.519615, accuracy: 0.968750
step: 1300, loss: 68.325279, accuracy: 0.949219
step: 1400, loss: 31.539482, accuracy: 0.964844
step: 1500, loss: 64.829033, accuracy: 0.925781
step: 1600, loss: 42.165142, accuracy: 0.949219
step: 1700, loss: 27.613605, accuracy: 0.968750
step: 1800, loss: 49.932446, accuracy: 0.945312
step: 1900, loss: 43.477921, accuracy: 0.937500
step: 2000, loss: 42.169315, accuracy: 0.945312
step: 2100, loss: 27.601894, accuracy: 0.968750
s